In [ ]:
import sys
import itertools
disjunction = lambda *args: any(args)

In [43]:
def find_pure(clauses, assignment):
    pure = {}
    deleted = []
    for clause in clauses:
        for lit in clause:
            if lit[0]=='~':
                c = lit[1:]
                if c in assignment:
                    continue
                
                elif c in pure:
                    if pure[c]!=False:
                        del pure[c]
                        deleted.append(c)

                elif c not in deleted:
                    pure[c]=False
                    
            else:
                if lit in assignment:
                    continue
                elif lit in pure:
                    if pure[lit]!=True:
                        del pure[lit]
                        deleted.append(lit)
                elif lit not in deleted:
                    pure[lit]=True
    
    #add the clause index where index = find(pure)
    '''listOfRemovals = []
    for j in range(len(clauses)):'''

    
    return pure

In [45]:
def isSat(clauses, assignment):
    for c in clauses:
        clause = []
        for literal in c:
            if literal[0]=='~':
                clause.append(not assignment[literal[1:]])
            else: 
                clause.append(assignment[literal])             
        #if any disjunction is False, the entire CNF expression will be false
        if disjunction(*clause) is False:
            return False
    return True

In [25]:
def unit_heuristic(clauses, assignment):
    clauses2 = clauses.copy()
    for i in range(len(clauses2)):
        clause = clauses2[i]
        if len(clause)==1:
            c = clause[0] 
            if c[0]=="~":
                #add to assignment if it does not exist, if it exists, then it must be from contradicting unit clauses and it is unsat
                if c[1:] in assignment and assignment[c[1:]]!=False:
                    #stands for unsatisfiable
                    return None, None 
                elif c[1:] not in assignment:
                    assignment[c[1:]] = False     
            
            else:
                if c in assignment and assignment[c]!=True:
                    return None, None #stands for unsatisfiable
                elif c not in assignment:
                    assignment[c] = True
            
            #remove clause
            clauses2[i] = None
    
    updated_clauses = [i for i in clauses2 if i is not None]
                    
    return assignment, updated_clauses

In [46]:
def extract(lines):
    literals = set()
    pure = dict() #assignments to pure
    clauses = []
    clause_idx = 0
    
    for line in lines:
    #each clause is a disjunction in cnf form
        clause = []
        for c in line.split(','):
            literals.add(c if c[0]!='~' else c[1:]) #add positive literal only
            clause.append(c)
        clauses.append(clause)

    return list(literals), clauses

In [36]:
def allLiteralsinAssign(clause, assignment):
    withoutsigns = []
    for lit in clause:
        if lit[0]=='~':
            if lit[1:] not in assignment:
                return False
        else:
            if lit not in assignment:
                return False
    return True
            


In [41]:
print(allLiteralsinAssign(['A','~B'],{'A':True, 'B':True}))

True


In [48]:
def isAnyClauseFalse(clauses, assignment):
    if len(clauses)==0 :
        return False
    if len(assignment)==0:
        return False
    
    #for each clause
    for clause in clauses:
        truth_val = []
        #check if all literals in clause are in assignment
        if allLiteralsinAssign(clause, assignment):
            for literal in clause:
                if literal[0]=='~':
                    truth_val.append(not assignment[literal[1:]])
                else: 
                    truth_val.append(assignment[literal])             
        #if any disjunction is False, the entire CNF expression will be false
            if disjunction(*truth_val) is False:
                return True #clause is False
    return False #cannot determine


In [47]:
#are all clauses true even if assignment is incomplete
def isClauseTrue(clauses, assignment):
    if len(clauses)==0 or len(assignment)==0:
        return None
    
    for clause in clauses:
        clause_isTrue = False
        
        for literal in clause:
            if literal[0]=='~':
                c = literal[1:]
                if c in assignment and assignment[c]==False:
                    clause_isTrue = True
                    break
            else:
                if literal in assignment and assignment[literal]==True:
                    clause_isTrue=True
                    break
                
        if clause_isTrue is False:
            return None
    return assignment

In [49]:
def DPLL(clauses, literals, model,length,doUnit=True, doPure=True):

    if len(literals)==0: #all assigned
        #find if model is satisfiable with current clauses #satisfiable should return True or False
        if isSat(clauses, model):
            return model
        else:
            return None
   
    else:
        #else, does the assignment entail True?
        entailTrue = isClauseTrue(clauses, model)
        if entailTrue:
            #assign the unassigned literals, here literals, to True
            return  {**model, **(dict.fromkeys(literals, True))}

        entailFalse = isAnyClauseFalse(clauses,model)
        if entailFalse:
            return None
        

    #if doUnit and only before recursion or first iteration
    if doUnit and model=={}:
        unit_assign, updated_clauses = unit_heuristic(clauses, {})
        if unit_assign is None:
            return None
        model.update(unit_assign)
        clauses = updated_clauses
    
    #only check pure for first iteration, since all pure would have been found the first time or as unit
    if doPure and model=={}:
        pure = find_pure(clauses, model)
        model.update(pure)


    #if all literals have been assigned
    if len(model)==length:
        if isSat(clauses,model):
            return model
        else:
            return None

    rest = [i for i in literals if i not in model.keys()]     #unassigned_keys
    first = rest.pop()


    
    return DPLL(clauses, rest, {**model, **{first:True}}, length) or DPLL(clauses, rest, {**model, **{first:False}}, length)



----

IGNORE
_______

In [ ]:

#find all possible combination of T/F values with Cartesian Product
def assign(literals):
    n = len(literals)
    for i in itertools.product([True,False], repeat=n):
        yield (dict(zip(literals, i)))

In [33]:
all(ele in ['A'] for ele in ['A','B'])

False

In [21]:
{**{'B':True}, **(dict.fromkeys(['A','C'], True))}

{'B': True, 'A': True, 'C': True}

In [26]:
sometestclauses=[['A'], ['~A']]
assign = {'A':True}

print(isClauseTrue(sometestclauses, assign))
print(unit_heuristic(sometestclauses, assign))



None
(None, None)


In [ ]:
#main solver
def cnf(lit, clauses):
    for assignment in assign(lit):
        if isSat(clauses, assignment) is True:
            print("satisfiable", end = " ")
            for key in assignment.keys():
                print(key+'='+str(assignment.get(key))[0],end=' ')
            print()
            return
    print('unsatisfiable')
    return
